In [1]:
import numpy as np
import torch
import warnings

from datasets import load_dataset
from transformers import AutoConfig, AutoModel
from hfmodel import MLPConfig, MLPHFModel, WRNConfig, WRNHFModel

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="`resume_download` is deprecated"
)

AutoConfig.register("mlp", MLPConfig)
AutoModel.register(MLPConfig, MLPHFModel)

AutoConfig.register("WideResNet", WRNConfig)
AutoModel.register(WRNConfig, WRNHFModel)



In [2]:
def _show_loss(dataset, model, device="cuda:0"):
    loss_fn=torch.nn.CrossEntropyLoss()
    model.to(device)
    model.eval()

    dataset.set_format("torch")
    data_loader = torch.utils.data.DataLoader(dataset, batch_size=len(dataset), shuffle=False)
    for batch in data_loader:
        pass
    all_logits = model(batch["feature"].to(device)).detach()
    loss = loss_fn(torch.Tensor(all_logits), batch["label"].to(device)).detach().cpu().numpy()
    return loss

def show_loss(dataset_name, model_name, model_idx=0):
    # Load dataset
    dataset_path = f"TDDBench/{dataset_name}"
    dataset = load_dataset(dataset_path)["train"]

    # Load target model
    model_path = f"TDDBench/{model_name}-{dataset_name}-{model_idx}"
    model = AutoModel.from_pretrained(model_path)

    # Load training data detection label, 1 means model's training data while 0 means model's non-training data
    config = AutoConfig.from_pretrained(model_path)
    tdd_label = np.array(config.tdd_label)

    # Calculate mean training loss and mean non-training loss
    train_idxes, nontrain_idxes = np.arange(len(tdd_label))[tdd_label==1], np.arange(len(tdd_label))[tdd_label==0]
    train_dataset = dataset.select(indices=train_idxes)
    nontrain_dataset = dataset.select(indices=train_idxes)

    print(f"Dataset {dataset_name}, Target model {model_idx}, Model index {model_idx}")
    mean_train_loss = _show_loss(train_dataset, model)
    print(f"Mean cross entropy loss of training samples: {mean_train_loss}")

    mean_nontrain_loss = _show_loss(nontrain_dataset, model)
    print(f"Mean cross entropy loss of non-training samples: {mean_nontrain_loss}")

In [3]:
!export HF_ENDPOINT=https://hf-mirror.com
show_loss(dataset_name="student", model_name="mlp")

Using the latest cached version of the dataset since TDDBench/student couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /data1/home/zhihao/.cache/huggingface/datasets/TDDBench___student/default/0.0.0/c14b433b016b8faa38a6976399b0bc3614afa131 (last modified on Wed Jul 23 14:41:57 2025).


Dataset student, Target model 0, Model index 0
Mean cross entropy loss of training samples: 0.40509146451950073
Mean cross entropy loss of non-training samples: 0.40509146451950073


In [4]:
show_loss(dataset_name="purchase100", model_name="mlp", model_idx=1)

Using the latest cached version of the dataset since TDDBench/purchase100 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /data1/home/zhihao/.cache/huggingface/datasets/TDDBench___purchase100/default/0.0.0/d7eaad068444a9b1d3366c698a2bddfd59dec4d6 (last modified on Wed Jul 23 15:09:19 2025).


Dataset purchase100, Target model 1, Model index 1
Mean cross entropy loss of training samples: 0.024750620126724243
Mean cross entropy loss of non-training samples: 0.024750620126724243
